In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
from astropy import units
from pathlib import Path
import os

import snapshot_obj
import dataset_compute
import curve_fit

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)
importlib.reload(curve_fit)

# Metallicity

Metallicity of an individual star is defined here as the mass fraction of the metals: $Z^*$ $ = \frac{ \sum_{j>\text{He}} m_j}{\sum_k m_k}$. Metallicity of a galaxy is defined as the mass weighted of average of the star particles of that galaxy: $Z = \frac{ \sum_j m_j Z^*_i}{\sum_k m_k}$ (in the EAGLE simulations). 

I further normalize the metallicity of a galaxy by the solar metallicity $Z_\odot = 0.0134$ (Wikipedia), and use the log-metallicity of the normalized quantity: $Z = \log_{10} \frac{ \sum_j m_j Z^*_i}{\sum_k m_k} - \log_{10} Z_\odot$.

## Construct data dictionary

Add entries for each simulation, and specify M31 and MW galaxies:

In [ ]:
snap_id = 127
sim_ids = ["V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["LCDM", "p082", "p084"]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (1,0)]

data = {}
for name, sim_id, m31_ns, mw_ns in zip(names, sim_ids, m31, mw):
    data[name] = {"snapshot": snapshot_obj.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns}

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = "by_gn"

Read datasets, split into satellites and isolated galaxies, and add to the data dictionary. We also disregard dark halos and potential spurious halos with $v_\mathrm{max} = 0$.

In [ ]:
metal_sun = np.log(0.0134)

for name, sim_data in data.items():
    # Get data:\frac{ \sum_j m_j Z^*_i}{\sum_k m_k}
    snap = sim_data["snapshot"]
    sm = snap.get_subhalos("Stars/Mass") * units.g.to(units.Msun)
    metal = np.log(snap.get_subhalos("Stars/Metallicity")) - metal_sun
    
    # Split into satellites:
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_gn":
        masks_sat, mask_isol = dataset_compute.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
        
    mask_lum, mask_dark = dataset_compute.split_luminous(snap)
    mask_nonzero_vmax = dataset_compute.prune_vmax(snap)

    # Add to dictionary:
    for dataset_name, dataset in zip(["SM", "Metallicity"],[sm, metal]):
        data[name][dataset_name] = \
        {"satellites": dataset[np.logical_and.reduce(
             [np.logical_or.reduce(masks_sat), mask_lum, mask_nonzero_vmax]
         )],
         "isolated": dataset[np.logical_and.reduce(
             [mask_isol, mask_lum, mask_nonzero_vmax]
         )]
        }

## Plot

In [ ]:
# Set some parameters:
x_down = 10**6; x_up = 10**11
y_down = -6; y_up = 1

# Set marker styles:
fcolor = ["black", "red", "blue", "green"]
mcolor = ["gray", "pink", "lightblue", "lightgreen"]
marker = ['+', "o", "^", 1]

In [ ]:
# Construct saving location:
filename = 'metallicity_{}'.format(distinction)
for name in names:
    filename += "_{}".format(name)
filename += ".png"
    
home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")
filename = os.path.join(path, filename)

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(14,6))
plt.subplots_adjust(wspace=0.3)

# Set axis:
for ax in axes:
    ax.set_xscale('log')
    ax.set_xlim(x_down, x_up)
    ax.set_ylim(y_down, y_up)    
    ax.set_xlabel('$M_*[\mathrm{M_\odot}]$', fontsize=16)
    ax.set_ylabel('$<[Z]>$', fontsize=16)

axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
for i, (name, entry) in enumerate(data.items()):
    x = entry['SM']['satellites']
    y = entry['Metallicity']['satellites']
    axes[0].scatter(x, y, s=20, marker=marker[i], c=mcolor[i], \
        edgecolor='none', label=name)
    
    x = entry['SM']['isolated']
    y = entry['Metallicity']['isolated']
    axes[1].scatter(x, y, s=20, marker=marker[i], c=mcolor[i], \
        edgecolor='none', label=name)
    
# Add median curves:
n_median_points = 7
for i, (name, entry) in enumerate(data.items()):
    x = entry['SM']['satellites']   
    y = entry['Metallicity']['satellites']
    print("# of satellites: {}".format(x.size))
    median = curve_fit.median_trend(x, y, n_points_per_bar=n_median_points)
    if median is not None:
        axes[0].plot(median[0], median[1], c=fcolor[i], linestyle='--')
    else:
        print("Could not fit median for:", name)
    
    x = entry['SM']['isolated']
    y = entry['Metallicity']['isolated']
    print("# of isolated galaxies: {}".format(x.size))
    median = curve_fit.median_trend(x, y, n_points_per_bar=n_median_points)
    if median is not None:
        axes[1].plot(median[0], median[1], c=fcolor[i], linestyle='--')
    else:
        print("Could not fit median for:", name)
    
axes[0].legend(loc='lower right')
plt.tight_layout()

#plt.savefig(filename, dpi=200)